# Uso de los modelos de aprendizaje automático

Jaime Alberto Suarez Moctezuma. 
Ejercicio #3.- 

In [41]:
#Importamos las bibliotecas necesarias. 
import pandas as pd
import requests

#Instalamos la biblioteca statsmodels.
!pip install statsmodels
#Descargamos el script graphing.py desde el respositorio de Github.
!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/graphing.py
#Especificamos la URL del conjunto de datos doggy-boot-harness.csv
url = 'https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/Data/doggy-boot-harness.csv'

#Obtenemos los datos del conjunto de datos a trabes de la URL. 
respuesta = requests.get(url)
# Guardar los datos descargados en un archivo local llamado doggy-boot-harness.csv
with open('doggy-boot-harness.csv', 'wb') as f:
    f.write(respuesta.content)

#Leemos los datos del archivo CSV recien creado usando pandas. 
datos = pd.read_csv('doggy-boot-harness.csv')
#Mostramos las primeras filas del Dataframe. 
datos.head()

Defaulting to user installation because normal site-packages is not writeable


"wget" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


,boot_size,harness_size,sex,age_years
0,39,58,male,12.0
1,38,58,male,9.6
2,37,52,female,8.6
3,39,58,male,10.2
4,38,57,male,7.8


In [47]:
# Importar la biblioteca statsmodels para ajuste de modelo.
import statsmodels.formula.api as smf

# Crear un modelo de regresión lineal ordinaria (OLS).
# La fórmula "boot_size ~ harness_size" especifica que estamos modelando boot_size en función de harness_size.
# 'data=datos' indica que los datos provienen del DataFrame llamado 'datos'.
modelo = smf.ols(formula="boot_size ~ harness_size", data=datos).fit()

# Imprimir un mensaje indicando que el modelo ha sido entrenado con éxito.
print("Modelo entrenado")

Modelo entrenado


In [48]:
# Importar la biblioteca joblib
import joblib

# Definir el nombre del archivo donde se guardará el modelo
nombre_archivo_modelo = './avalanche_dog_boot_model.pkl'
# Utilizar joblib para guardar el modelo en el archivo especificado
joblib.dump(modelo, nombre_archivo_modelo)

# Imprimir un mensaje indicando que el modelo ha sido guardado con éxito
print("Modelo Guardado")

Modelo Guardado


In [49]:
# Cargar el modelo desde el archivo y mostrar sus parámetros
modelo_cargado = joblib.load(nombre_archivo_modelo)

# Imprimimos el siguiente mensaje.
print ("Hemos cargado un modelo con los siguientes parámetros:")
print (modelo_cargado.params)

Hemos cargado un modelo con los siguientes parámetros:
Intercept       5.719110
harness_size    0.585925
dtype: float64


In [50]:
def modelo_cargado_para_predecir(harness_size):
    '''
    Esta función carga un modelo previamente entrenado. Utiliza el modelo
    con el tamaño del arnés del perro del cliente para predecir el tamaño de
    botas que le queden bien a ese perro.

    Harness_size: El tamaño del arnés del perro, en cm.
    '''
    
    # Cargar el modelo previamente guardado
    Carga_modelo = joblib.load(nombre_archivo_modelo)
    
    # Imprimir los parámetros del modelo cargado
    print("Hemos cargado un modelo con los siguientes parámetros:")
    print(Carga_modelo.params)
    
    # Predecir el tamaño de las botas para el arnés proporcionado
    inputs = {"harness_size":[harness_size]}
    predicción_tamaño_botas = Carga_modelo.predict(inputs)[0]
    
    return predicción_tamaño_botas

# Uso de la función.
predicción_tamaño_botas = modelo_cargado_para_predecir(45)
print("Tamaño previsto de la bota para perros:", predicción_tamaño_botas)

Hemos cargado un modelo con los siguientes parámetros:
Intercept       5.719110
harness_size    0.585925
dtype: float64
Tamaño previsto de la bota para perros: 32.08575356590478


In [51]:
def comprobar_tamaño_de_botas(tamaño_seleccionado_arnés , tamaño_seleccionado_botas): 
    '''
    Calcula si el cliente ha elegido un par de botas para perros que
    son de un tamaño razonable. Esto funciona estimando la bota real del perro.
    tamaño del tamaño de su arnés.

    Esto devuelve un mensaje para el cliente que debe mostrarse antes
    completan su pago

    selected_harness_size: El tamaño del arnés que el cliente quiere comprar
    selected_boot_size: El tamaño de las botas para perros que el cliente quiere comprar.
    '''
    
    # Estimar el tamaño de las botas según el arnés seleccionado
    tamaño_arranque_estimado = modelo_cargado_para_predecir(tamaño_seleccionado_arnés)
    tamaño_arranque_estimado = int(round(tamaño_arranque_estimado))
    
    # Comprobar si el tamaño seleccionado es igual al estimado
    if tamaño_seleccionado_botas == tamaño_arranque_estimado:
        return f"¡Gran elección! Creemos que estas botas le quedarán bien a su perro de avalanchas."
    
    # Comprobar si el tamaño seleccionado es menor al estimado
    if tamaño_seleccionado_botas < tamaño_arranque_estimado:
        return "El modelo seleccionado pueden ser DEMASIADO PEQUEÑO para un perro como"\
               f"tan grande como el tuyo. Recomendamos una talla de botas para perros de {tamaño_arranque_estimado}."
    
    # Comprobar si el tamaño seleccionado es mayor al estimado
    if tamaño_seleccionado_botas > tamaño_arranque_estimado: 
            return "Las botas que has seleccionado pueden ser DEMASIADO GRANDES para un perro como "\
                   f"pequeñas como las tuyas. Recomendamos una talla de botas para perros de {tamaño_arranque_estimado}."
    
# Ejemplo de uso de la función.     
comprobar_tamaño_de_botas(tamaño_seleccionado_arnés=55, tamaño_seleccionado_botas=39)

Hemos cargado un modelo con los siguientes parámetros:
Intercept       5.719110
harness_size    0.585925
dtype: float64


'Las botas que has seleccionado pueden ser DEMASIADO GRANDES para un perro como pequeñas como las tuyas. Recomendamos una talla de botas para perros de 38.'